# Детекция переломов на рентген‑снимках: подготовка данных и обучение

Этот ноутбук фиксирует ключевые шаги проекта: подготовку данных, объединение классов, конвертацию YOLO‑разметки в COCO,
обучение Faster R‑CNN в Detectron2 и базовое обучение YOLO.
Это копия моего kaggle ноутбука с исправлениями от нейронки(комментарии)

## 0. Установка зависимостей (опционально)

Если запускаете в чистом окружении, установите зависимости. В проекте используются: torch/torchvision, detectron2, ultralytics, pycocotools.


In [ ]:
# Если нужно — установите зависимости (можно запускать по одной строке)
# !pip install -r requirements.txt
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


## 1. Импорты и версии

Проверяем, что основные библиотеки доступны и видим их версии.


In [ ]:
import torch

try:
    import detectron2
    detectron2_version = detectron2.__version__
except Exception as exc:
    detectron2_version = f"not available ({exc})"

try:
    from ultralytics import YOLO
    ultralytics_ok = True
except Exception as exc:
    YOLO = None
    ultralytics_ok = False
    ultralytics_error = exc

print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("detectron2:", detectron2_version)
print("ultralytics:", "ok" if ultralytics_ok else f"not available ({ultralytics_error})")


## 2. Пути к данным

Ожидаем структуру данных как в оригинальном датасете YOLO: `train/valid/test` + `images/labels`.
Данные не хранятся в репозитории — скачиваются отдельно и кладутся в `data/BoneFractureYolo8`.


In [ ]:
from pathlib import Path

DATA_ROOT = Path("data/BoneFractureYolo8")

TRAIN_IMG = DATA_ROOT / "train" / "images"
TRAIN_LBL = DATA_ROOT / "train" / "labels"

VAL_IMG = DATA_ROOT / "valid" / "images"
VAL_LBL = DATA_ROOT / "valid" / "labels"

TEST_IMG = DATA_ROOT / "test" / "images"
TEST_LBL = DATA_ROOT / "test" / "labels"

# JSON‑файлы после конвертации в COCO (для Detectron2 и метрик)
TRAIN_JSON = DATA_ROOT / "train_merged.json"
VAL_JSON = DATA_ROOT / "val_merged.json"
TEST_JSON = DATA_ROOT / "test_merged.json"

# Оригинальный data.yaml для YOLO (7 классов)
DATA_YAML = DATA_ROOT / "data.yaml"


## 3. Классы и объединение

В исходной разметке 7 классов. Класс **humerus fracture** содержит всего 3 объекта,
поэтому он объединён с **humerus**. Итоговый протокол — 6 классов.

Также в данных присутствуют изображения без переломов, и они использовались при обучении.


In [ ]:
# old ids: 0..6
OLD_NAMES = [
    "elbow positive",
    "fingers positive",
    "forearm fracture",
    "humerus fracture",
    "humerus",
    "shoulder fracture",
    "wrist positive",
]

# merge old 3 -> old 4
MERGE_FROM_OLD = 3
MERGE_TO_OLD = 4

# keep these old ids after merge
KEPT_OLD_IDS = [0, 1, 2, 4, 5, 6]
OLD_TO_NEW = {old_id: new_id for new_id, old_id in enumerate(KEPT_OLD_IDS)}

NEW_NAMES = [
    "elbow positive",
    "fingers positive",
    "forearm fracture",
    "humerus",
    "shoulder fracture",
    "wrist positive",
]


## 4. Краткий анализ разметки (YOLO‑сегментации)

Считаем количество объектов по классам и долю пустых изображений (без разметки).
Это помогает принять решения по объединению классов и параметрам детекции.


In [ ]:
from collections import Counter, defaultdict
import numpy as np


def scan_yolo_seg_labels(labels_dir: Path, num_classes: int):
    labels_dir = Path(labels_dir)
    txts = sorted(labels_dir.rglob("*.txt"))

    inst_per_class = Counter()
    imgs_with_class = Counter()
    empty_images = 0

    areas_by_class = defaultdict(list)
    wh_by_class = defaultdict(list)

    for txt in txts:
        lines = [l.strip() for l in txt.read_text().splitlines() if l.strip()]
        if not lines:
            empty_images += 1
            continue

        present = set()

        for line in lines:
            parts = line.split()
            if len(parts) < 7:
                continue

            cls = int(float(parts[0]))
            if not (0 <= cls < num_classes):
                continue

            coords = list(map(float, parts[1:]))
            if len(coords) % 2 != 0:
                continue

            xs = np.array(coords[0::2], dtype=np.float32)
            ys = np.array(coords[1::2], dtype=np.float32)

            xmin, xmax = float(xs.min()), float(xs.max())
            ymin, ymax = float(ys.min()), float(ys.max())
            w = max(0.0, xmax - xmin)
            h = max(0.0, ymax - ymin)
            area = w * h

            inst_per_class[cls] += 1
            present.add(cls)

            wh_by_class[cls].append((w, h))
            areas_by_class[cls].append(area)

        for c in present:
            imgs_with_class[c] += 1

    total_images = len(txts)

    print(f"Всего label‑файлов (≈ изображений): {total_images}")
    print(f"Пустых (нет разметки): {empty_images}  ({empty_images/total_images:.1%})
")

    print("Объектов (instances) по классам:")
    for c in range(num_classes):
        print(f"  class {c}: {inst_per_class[c]}")

    print("
Изображений, где встречается класс:")
    for c in range(num_classes):
        print(f"  class {c}: {imgs_with_class[c]}  ({imgs_with_class[c]/total_images:.1%})")

    print("
Размеры bbox (нормализованные), медианы по классам:")
    for c in range(num_classes):
        if len(wh_by_class[c]) == 0:
            print(f"  class {c}: нет объектов")
            continue
        ws = np.array([x[0] for x in wh_by_class[c]])
        hs = np.array([x[1] for x in wh_by_class[c]])
        ars = np.array(areas_by_class[c])
        print(
            f"  class {c}: median w={np.median(ws):.4f}, "
            f"h={np.median(hs):.4f}, area={np.median(ars):.6f}"
        )


scan_yolo_seg_labels(labels_dir=TRAIN_LBL, num_classes=7)


## 5. Конвертация YOLO‑сегментаций → COCO (bbox) с объединением классов

Конвертация нужна для Detectron2 и для сравнимых метрик.
Выходные файлы: `train_merged.json`, `val_merged.json`, `test_merged.json`.


In [ ]:
import json
from PIL import Image


def yolo_seg_to_coco_merged(images_dir: Path, labels_dir: Path, out_json: Path):
    images_dir = Path(images_dir)
    labels_dir = Path(labels_dir)

    coco = {
        "images": [],
        "annotations": [],
        "categories": [{"id": i, "name": n} for i, n in enumerate(NEW_NAMES)],
    }

    ann_id = 1
    img_id = 1

    img_paths = []
    for ext in ("*.png", "*.jpg", "*.jpeg", "*.bmp", "*.tif", "*.tiff"):
        img_paths += list(images_dir.rglob(ext))
    img_paths = sorted(img_paths)

    bad_class_lines = 0

    for img_path in img_paths:
        with Image.open(img_path) as im:
            w, h = im.size

        coco["images"].append({
            "id": img_id,
            "file_name": str(img_path.relative_to(images_dir)),
            "width": w,
            "height": h,
        })

        label_path = labels_dir / (img_path.stem + ".txt")
        lines = []
        if label_path.exists():
            lines = [l.strip() for l in label_path.read_text().splitlines() if l.strip()]

        for line in lines:
            parts = line.split()
            if len(parts) < 1 + 6:
                continue

            cls_old = int(float(parts[0]))
            if not (0 <= cls_old < len(OLD_NAMES)):
                bad_class_lines += 1
                continue

            # merge 3 -> 4
            if cls_old == MERGE_FROM_OLD:
                cls_old = MERGE_TO_OLD

            # пропускаем удалённый класс
            if cls_old not in OLD_TO_NEW:
                continue

            cls_new = OLD_TO_NEW[cls_old]

            coords = list(map(float, parts[1:]))
            if len(coords) % 2 != 0:
                continue

            xs = coords[0::2]
            ys = coords[1::2]

            xs_px = [max(0.0, min(w - 1.0, x * w)) for x in xs]
            ys_px = [max(0.0, min(h - 1.0, y * h)) for y in ys]

            xmin, xmax = min(xs_px), max(xs_px)
            ymin, ymax = min(ys_px), max(ys_px)

            bw = max(1.0, xmax - xmin)
            bh = max(1.0, ymax - ymin)

            bbox = [float(xmin), float(ymin), float(bw), float(bh)]

            poly = []
            for x, y in zip(xs_px, ys_px):
                poly += [float(x), float(y)]
            seg = [poly] if len(poly) >= 6 else []

            coco["annotations"].append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": cls_new,
                "bbox": bbox,
                "area": float(bw * bh),
                "iscrowd": 0,
                "segmentation": seg,
            })
            ann_id += 1

        img_id += 1

    out_json = Path(out_json)
    out_json.parent.mkdir(parents=True, exist_ok=True)
    out_json.write_text(json.dumps(coco, ensure_ascii=False))

    print(f"Saved {out_json}: images={len(coco['images'])}, anns={len(coco['annotations'])}")
    if bad_class_lines:
        print(f"WARNING: skipped {bad_class_lines} label lines with class_id outside [0..6]")


yolo_seg_to_coco_merged(TRAIN_IMG, TRAIN_LBL, TRAIN_JSON)
yolo_seg_to_coco_merged(VAL_IMG, VAL_LBL, VAL_JSON)
yolo_seg_to_coco_merged(TEST_IMG, TEST_LBL, TEST_JSON)


## 6. Регистрация датасетов в Detectron2

Регистрируем COCO‑json и фиксируем классы в metadata.


In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

register_coco_instances("fracture_train_merged", {}, str(TRAIN_JSON), str(TRAIN_IMG))
register_coco_instances("fracture_val_merged", {}, str(VAL_JSON), str(VAL_IMG))

MetadataCatalog.get("fracture_train_merged").thing_classes = NEW_NAMES
MetadataCatalog.get("fracture_val_merged").thing_classes = NEW_NAMES

print("Registered merged datasets with classes:", NEW_NAMES)


## 7. Обучение Faster R‑CNN (Detectron2)

Базовый конфиг R50‑FPN. Ключевые параметры: `NUM_CLASSES=6`, `imgsz=1024`, `FILTER_EMPTY_ANNOTATIONS=False`,
расширенные `aspect ratios` для вытянутых боксов.

Данные уже содержат аугментации (поворот, яркость/контраст). Дополнительные аугментации заметного улучшения не дали.


In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
import os


class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        os.makedirs(output_folder, exist_ok=True)
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("fracture_train_merged",)
cfg.DATASETS.TEST = ("fracture_val_merged",)

cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6


cfg.INPUT.MIN_SIZE_TRAIN = (1024,)
cfg.INPUT.MAX_SIZE_TRAIN = 1024
cfg.INPUT.MIN_SIZE_TEST = 1024
cfg.INPUT.MAX_SIZE_TEST = 1024

cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.33, 0.5, 1.0, 2.0, 3.0]] * 5
cfg.TEST.DETECTIONS_PER_IMAGE = 300

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.MAX_ITER = 15000
cfg.SOLVER.STEPS = (10000, 13000)
cfg.SOLVER.GAMMA = 0.1

cfg.OUTPUT_DIR = str(DATA_ROOT / "frcnn_r50_baseline")
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


## 8. Визуализация предсказаний Faster R‑CNN

Проверка качества боксов на валидации.


In [ ]:
import random
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg.MODEL.WEIGHTS = str(Path(cfg.OUTPUT_DIR) / "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
predictor = DefaultPredictor(cfg)

dataset_dicts = DatasetCatalog.get("fracture_val_merged")
metadata = MetadataCatalog.get("fracture_val_merged")

for d in random.sample(dataset_dicts, k=min(6, len(dataset_dicts))):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(10, 10))
    plt.imshow(out.get_image())
    plt.axis("off")
    plt.show()


## 9. Обучение YOLO

YOLO обучается на оригинальном датасете (7 классов) по `data.yaml`.
Использовались параметры `epochs=50` и `imgsz=640`.


In [ ]:
if YOLO is None:
    raise RuntimeError("Ultralytics не установлен")

model = YOLO("yolo26m.pt")
model.train(data=str(DATA_YAML), epochs=50, imgsz=640, device=[0, 1])
